# Import Libraries

In [1]:
import sys
sys.path.append('../../gemgis')
import geopandas as gpd
import rasterio
import gemgis as gg

print(gg)

<module 'gemgis' from '../../gemgis\\gemgis.py'>


# Read Vector Files - Polygons

In [2]:
gdf = gpd.read_file('data/vector_data/GeologicalMapAachen.shp')

In [3]:
gdf.crs

{'init': 'epsg:4647'}

In [4]:
gdf.geom_type

0      Polygon
1      Polygon
2      Polygon
3      Polygon
4      Polygon
        ...   
843    Polygon
844    Polygon
845    Polygon
846    Polygon
847    Polygon
Length: 848, dtype: object

# Read Vector Files - Points

In [5]:
interfaces = gpd.read_file('C:/Users/user_abuser/Desktop/Kartenkunde/interfaces.shp')

In [6]:
interfaces.crs

{'init': 'epsg:4326'}

In [7]:
interfaces.geom_type

0     Point
1     Point
2     Point
3     Point
4     Point
5     Point
6     Point
7     Point
8     Point
9     Point
10    Point
11    Point
12    Point
13    Point
14    Point
15    Point
16    Point
17    Point
18    Point
19    Point
20    Point
21    Point
22    Point
23    Point
24    Point
25    Point
26    Point
27    Point
28    Point
29    Point
30    Point
31    Point
32    Point
33    Point
34    Point
35    Point
36    Point
37    Point
38    Point
39    Point
40    Point
dtype: object

In [8]:
interfaces.head()

,id,formation,geometry
0,None,Ton,POINT (19.150 293.313)
1,None,Ton,POINT (61.934 381.459)
2,None,Ton,POINT (109.358 480.946)
3,None,Ton,POINT (157.812 615.999)
4,None,Ton,POINT (191.318 719.094)


## Extract X,Y coordinates

In [9]:
interfaces = gg.extract_xy_values(interfaces)
interfaces.head()

,id,formation,geometry,X,Y
0,None,Ton,POINT (19.150 293.313),19.150128,293.313485
1,None,Ton,POINT (61.934 381.459),61.934367,381.459326
2,None,Ton,POINT (109.358 480.946),109.357860,480.945568
3,None,Ton,POINT (157.812 615.999),157.812299,615.999430
4,None,Ton,POINT (191.318 719.094),191.318028,719.093981


# Load Vector Data - LineString

In [10]:
topo = gpd.read_file('C:/Users/user_abuser/Desktop/Kartenkunde/Topography.shp')

In [11]:
topo.head()

,id,Z,geometry
0,None,400,"LINESTRING (0.741 475.441, 35.629 429.247, 77...."
1,None,300,"LINESTRING (645.965 0.525, 685.141 61.866, 724..."
2,None,400,"LINESTRING (490.292 0.525, 505.756 40.732, 519..."
3,None,600,"LINESTRING (911.433 1068.585, 908.856 1026.831..."
4,None,700,"LINESTRING (228.432 1068.585, 239.772 1017.037..."


## Extract X,Y coordinates

In [12]:
topo = gg.extract_xy_values(topo)
topo.head()

,id,Z,geometry,points,X,Y
0,None,400,"LINESTRING (0.741 475.441, 35.629 429.247, 77....","(0.7408806771479846, 475.44101474698454)",0.740881,475.441015
0,None,400,"LINESTRING (0.741 475.441, 35.629 429.247, 77....","(35.62873136073459, 429.2469161566801)",35.628731,429.246916
0,None,400,"LINESTRING (0.741 475.441, 35.629 429.247, 77....","(77.30033078835194, 340.0890755208477)",77.300331,340.089076
0,None,400,"LINESTRING (0.741 475.441, 35.629 429.247, 77....","(104.75836141895252, 269.34426719024157)",104.758361,269.344267
0,None,400,"LINESTRING (0.741 475.441, 35.629 429.247, 77....","(127.04782157791061, 207.64445718500974)",127.047822,207.644457


# Read Digital Elevation Model

In [33]:
dem = rasterio.open('C:/Users/user_abuser/Desktop/Kartenkunde/Task1_modifiziert.tif')
dem

<open DatasetReader name='C:/Users/user_abuser/Desktop/Kartenkunde/Task1_modifiziert.tif' mode='r'>

In [34]:
interfaces = gpd.read_file('C:/Users/user_abuser/Desktop/Kartenkunde/interfaces.shp')

In [35]:
interfaces = gg.extract_z_values(interfaces, dem)
interfaces.head()

,id,formation,geometry,X,Y,Z
0,None,Ton,POINT (19.150 293.313),19.150128,293.313485,115
1,None,Ton,POINT (61.934 381.459),61.934367,381.459326,254
2,None,Ton,POINT (109.358 480.946),109.357860,480.945568,61
3,None,Ton,POINT (157.812 615.999),157.812299,615.999430,24
4,None,Ton,POINT (191.318 719.094),191.318028,719.093981,0


# Save as GemPy df

In [37]:
interfaces = gg.convert_to_gempy_df(interfaces)
interfaces.head()

,X,Y,Z,formation
0,19.150128,293.313485,115,Ton
1,61.934367,381.459326,254,Ton
2,109.357860,480.945568,61,Ton
3,157.812299,615.999430,24,Ton
4,191.318028,719.093981,0,Ton


In [40]:
interfaces['dip'] = 50
interfaces['azimuth'] = 100

In [41]:
orientations= gg.convert_to_gempy_df(interfaces)
orientations.head()

,X,Y,Z,formation,dip,azimuth,polarity
0,19.150128,293.313485,115,Ton,50,100,1
1,61.934367,381.459326,254,Ton,50,100,1
2,109.357860,480.945568,61,Ton,50,100,1
3,157.812299,615.999430,24,Ton,50,100,1
4,191.318028,719.093981,0,Ton,50,100,1


# Define Custom Sections

In [50]:
custom_sections = gpd.read_file('C:/Users/user_abuser/Desktop/Kartenkunde/Custom_Section_1.shp')

In [51]:
custom_sections

,id,geometry
0,None,POINT (695.467 3.226)
1,None,POINT (669.284 1060.822)


In [55]:
custom_sections['section_name'] = 'Section1'
custom_sections

,id,geometry,section_name,X,Y
0,None,POINT (695.467 3.226),Section1,695.466746,3.226225
1,None,POINT (669.284 1060.822),Section1,669.284003,1060.822026


In [56]:
section_dict = gg.to_section_dict(custom_sections, 'section_name')
section_dict

{'Section1': ([695.4667461080886, 3.2262250771374283],
  [[669.2840030245482, 1060.822026058724]],
  [100, 80])}

In [57]:
import pandas as pd
sections = pd.concat([custom_sections,custom_sections,custom_sections])
sections['section_name'].iloc[2] ='Section2'
sections['section_name'].iloc[3] ='Section2'
sections['section_name'].iloc[4] ='Section3'
sections['section_name'].iloc[5] ='Section3'
sections

C:\Users\user_abuser\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,id,geometry,section_name,X,Y
0,None,POINT (695.467 3.226),Section1,695.466746,3.226225
1,None,POINT (669.284 1060.822),Section1,669.284003,1060.822026
0,None,POINT (695.467 3.226),Section2,695.466746,3.226225
1,None,POINT (669.284 1060.822),Section2,669.284003,1060.822026
0,None,POINT (695.467 3.226),Section3,695.466746,3.226225
1,None,POINT (669.284 1060.822),Section3,669.284003,1060.822026


In [58]:
section_dict = gg.to_section_dict(sections, 'section_name')
section_dict

{'Section1': ([695.4667461080886, 3.2262250771374283],
  [[669.2840030245482, 1060.822026058724]],
  [100, 80]),
 'Section2': ([695.4667461080886, 3.2262250771374283],
  [[669.2840030245482, 1060.822026058724]],
  [100, 80]),
 'Section3': ([695.4667461080886, 3.2262250771374283],
  [[669.2840030245482, 1060.822026058724]],
  [100, 80])}

# Create GemPyData Object

In [42]:
data = gg.GemPyData()
print(data.crs)

None


In [43]:
data.crs = 'EPSG:4326'
data.crs

'EPSG:4326'

In [44]:
data = gg.GemPyData(crs = 'EPSG:4326')
print(data.crs)

EPSG:4326


In [48]:
data.interfaces = interfaces
data.interfaces.head()

,X,Y,Z,formation,dip,azimuth
0,19.150128,293.313485,115,Ton,50,100
1,61.934367,381.459326,254,Ton,50,100
2,109.357860,480.945568,61,Ton,50,100
3,157.812299,615.999430,24,Ton,50,100
4,191.318028,719.093981,0,Ton,50,100


In [49]:
data.orientations = orientations
data.orientations.head()

,X,Y,Z,formation,dip,azimuth,polarity
0,19.150128,293.313485,115,Ton,50,100,1
1,61.934367,381.459326,254,Ton,50,100,1
2,109.357860,480.945568,61,Ton,50,100,1
3,157.812299,615.999430,24,Ton,50,100,1
4,191.318028,719.093981,0,Ton,50,100,1


In [59]:
data.section_dict = section_dict
data.section_dict

{'Section1': ([695.4667461080886, 3.2262250771374283],
  [[669.2840030245482, 1060.822026058724]],
  [100, 80]),
 'Section2': ([695.4667461080886, 3.2262250771374283],
  [[669.2840030245482, 1060.822026058724]],
  [100, 80]),
 'Section3': ([695.4667461080886, 3.2262250771374283],
  [[669.2840030245482, 1060.822026058724]],
  [100, 80])}